In [15]:
import pandas as pd
from MMRcall import MMRapi
from Cnx import Cnx
from fuzzywuzzy import fuzz
import make as mk
import difflib

"""
Platterfunc takes in a batch of VIN numbers and serves up
a piping hot set of residual values.
"""

# STATIC VARS
# ...

def main(vin):
    """
    Input: VIN.
    Output: Dict of residuals with axes: 
    {Make, Model, Trim, Term_1,..., Term_n, MileageBand_1,..., MileageBand_n}
    """
    conn = Cnx()
    mmr = MMRapi(vin)
    mmr.match()
    make = mmr.make()
    bod = mmr.body()
    model = mmr.model()
    year = mmr.model_year()
    """Access MMR Api"""
    slice_bymake = f"""
    SELECT [ALGResidualNewID] ,[ModelYear] ,[MakeNumber] ,[ModelNumber] ,[Style] ,[ModelDesc] ,[Description] ,[Month24] ,[Month30] ,[Month36] ,[Month42] ,[Month48] ,[Month54] ,[Month60] FROM [vinput].[dbo].[ALGResidualNewTable20220708]
    WHERE MakeNumber = {mk.make_number[make]} AND
    ModelYear = {year}
    """
    conn.set_query(slice_bymake)
    make_match = conn.execute()
    df = pd.DataFrame(make_match)
    df.columns = [ 'ALGResidualNewID', 'ModelYear', 'MakeNumber', 'ModelNumber', 'Style', 'ModelDesc', 'Description', 'Month24', 'Month30', 'Month36', 'Month42', 'Month48', 'Month54', 'Month60' ]

    best_fit = difflib.SequenceMatcher()
    best_fit.set_seq1(bod)
    def find_it(x):
        best_fit.set_seq2(x)
        return best_fit.ratio()
    
    df['Fuzzy'] = df[:]['Description'].map(lambda x: find_it(x))
    scope = df.loc[df.Fuzzy == df.Fuzzy.max()]
    


    
    
main('5UXCR6C09N9M97942')

In [16]:
s = difflib.SequenceMatcher(None, " im just affpples to appffffle", "apples to apples").ratio()
s

0.6521739130434783

In [24]:
import pandas as pd
from MMRcall import MMRapi
from Cnx import Cnx
from fuzzywuzzy import fuzz
import make as mk
import difflib

"""
Platterfunc takes in a batch of VIN numbers and serves up
a piping hot set of residual values.
"""

# STATIC VARS
# ...

def main(vin):
    """
    Input: VIN.
    Output: Dict of residuals with axes: 
    {Make, Model, Trim, Term_1,..., Term_n, MileageBand_1,..., MileageBand_n}
    """
    conn = Cnx()
    mmr = MMRapi(vin)
    mmr.match()
    make = mmr.make()
    bod = mmr.body()
    model = mmr.model()
    year = mmr.model_year()
    """Access MMR Api"""

    if make == 'Ford' or make == 'Chevrolet' or make == 'Dodge':
        slice_bymake = f"""
            SELECT [ALGResidualNewID] ,[ModelYear] ,[MakeNumber] ,[ModelNumber] ,[Style] ,[ModelDesc] ,[Description], [Region] ,[Month24] ,[Month30] ,[Month36] ,[Month42] ,[Month48] ,[Month54] ,[Month60] FROM [vinput].[dbo].[ALGResidualNewTable20220708]
            WHERE MakeNumber = {mk.make_number[make][0]} or MakeNumber = {mk.make_number[make][1]}  AND
            ModelYear = {year}
        """
    else:
        slice_bymake = f"""
            SELECT [ALGResidualNewID] ,[ModelYear] ,[MakeNumber] ,[ModelNumber] ,[Style] ,[ModelDesc] ,[Description], [Region] ,[Month24] ,[Month30] ,[Month36] ,[Month42] ,[Month48] ,[Month54] ,[Month60] FROM [vinput].[dbo].[ALGResidualNewTable20220708]
            WHERE MakeNumber = {mk.make_number[make]} AND
            ModelYear = {year}
        """
    
    conn.set_query(slice_bymake)
    make_match = conn.execute()
    df = pd.DataFrame(make_match)
    df.columns = [ 'ALGResidualNewID', 'ModelYear', 'MakeNumber', 'ModelNumber', 'Style', 'ModelDesc', 'Description', 'Region', 'Month24', 'Month30', 'Month36', 'Month42', 'Month48', 'Month54', 'Month60' ]

    def find_it(x):
        return fuzz.partial_ratio(bod, x)


    best_fit = difflib.SequenceMatcher()
    best_fit.set_seq1(bod)
    def find_it_fast(x):
        best_fit.set_seq2(x)
        return best_fit.real_quick_ratio()
    
    df['Fuzzy'] = df[:]['Description'].map(lambda x: find_it_fast(x))
    
    df = df[df.ModelDesc == model]
    df = df[df.Fuzzy == df.Fuzzy.max()]
    return df

    


    
    
main('5UXCR6C09N9M97942')

,ALGResidualNewID,ModelYear,MakeNumber,ModelNumber,Style,ModelDesc,Description,Region,Month24,Month30,Month36,Month42,Month48,Month54,Month60,Fuzzy
64,8085851,2022,100,180,110,X5,4dr Wgn sDrive40i,N,33,31,29,25,22,14,10,0.607143
65,8085852,2022,100,180,115,X5,4dr Wgn xDrive40i,N,35,33,31,27,24,16,10,0.607143
66,8085853,2022,100,180,132,X5,4dr Wgn xDrive45e,N,36,34,32,28,25,17,10,0.607143
74,8091737,2022,100,180,110,X5,4dr Wgn sDrive40i,A,33,31,29,25,22,14,10,0.607143
75,8091738,2022,100,180,115,X5,4dr Wgn xDrive40i,A,35,33,31,27,24,16,10,0.607143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12632,8789183,2022,100,180,132,X5,4dr Wgn xDrive45e,C,36,34,32,27,24,16,10,0.607143
12641,9026116,2022,100,180,132,X5,4dr Wgn xDrive45e,N,36,34,32,26,23,16,10,0.607143
12678,9045430,2022,100,180,110,X5,4dr Wgn sDrive40i,A,32,30,28,22,19,12,10,0.607143
12679,9045431,2022,100,180,115,X5,4dr Wgn xDrive40i,A,34,32,30,24,21,14,10,0.607143


In [25]:
import pandas as pd

mapp = pd.read_csv('ALG_US_CHROMEMAP.csv')
mein_map = mapp[mapp.ChromeStyleId == ]

,Country,EffectiveDate,ModelYear,AlgCode,ChromeStyleId,IsPrimary
0,US,2022-07-01,2023,20125138,429173,1
1,US,2022-07-01,2023,20125145,429174,1
2,US,2022-07-01,2023,20125150,429175,1
3,US,2022-07-01,2023,20125155,429176,1
4,US,2022-07-01,2023,40150110,427828,1
...,...,...,...,...,...,...
6796,US,2022-07-01,2021,860400180,413930,1
6797,US,2022-07-01,2021,860400183,413931,1
6798,US,2022-07-01,2021,860400185,413933,1
6799,US,2022-07-01,2021,860400186,413934,1
